In [1]:
# 1. Instalar Tesseract OCR y su soporte en Python
print("Instalando Tesseract OCR y PyTesseract...")
!sudo apt install tesseract-ocr
!pip install pytesseract
!sudo apt-get install tesseract-ocr-spa # Instalar soporte para español

# 2. Instalar librerías de Hugging Face (TTS) y manejo de audio
print("Instalando librerías de Hugging Face (TTS) y Audio...")
!pip install transformers torch soundfile librosa

# Librería para manejo de imágenes
!pip install Pillow

# Asegurar que el soporte de Tesseract para español sea accesible
import os
os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata'

print("\n Instalación completa.")

Instalando Tesseract OCR y PyTesseract...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-spa is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Instalando librerías de Hugging Face (TTS) y Audio...

 Instalación completa.


In [2]:
## --- PARTE 1: RECONOCIMIENTO DE TEXTO (OCR con Tesseract) ---

import pytesseract
from PIL import Image

# 1. Definir la ruta de la imagen
image_path = "ejemplo2.jpg"

try:
    image = Image.open(image_path)
    print(f"Imagen '{image_path}' cargada correctamente.")
except FileNotFoundError:
    print(f"❌ ERROR: No se encontró la imagen en la ruta: {image_path}")
    print("Asegúrate de haber subido el archivo a Colab y de que el nombre de la ruta sea correcto.")
    raise

# 2. Transcribir la Imagen usando Tesseract con soporte en español
transcribed_text = pytesseract.image_to_string(image, lang='spa').strip()

# 3. Limpieza básica del resultado del OCR
# se quitan saltos de línea y espacios extra
transcribed_text = ' '.join(transcribed_text.split())

print("\n--- RESULTADO OCR ---")
print(f"Texto Transcrito Bruto: '{transcribed_text}'")

Imagen 'ejemplo2.jpg' cargada correctamente.

--- RESULTADO OCR ---
Texto Transcrito Bruto: 'Hola, esto es un texto de prueba para el Convertidor de texto a audio.'


In [3]:
## --- PARTE 2: POST-PROCESAMIENTO Y LIMPIEZA ---

# 1. Resultado de la transcripción
text_to_clean = transcribed_text

# 2. Corrección de Puntuación
if text_to_clean and not text_to_clean.endswith(('?', '.', '!', ':', ';')):
    corrected_text = text_to_clean.strip() + '?'
else:
    corrected_text = text_to_clean.strip()

print(f"\n Texto Limpio y Preparado para TTS: '{corrected_text}'")


 Texto Limpio y Preparado para TTS: 'Hola, esto es un texto de prueba para el Convertidor de texto a audio.'


In [4]:
## --- PARTE 3: SÍNTESIS DE VOZ (TTS) ---

import torch
import soundfile as sf
from transformers import VitsModel, AutoTokenizer
from IPython.display import Audio

# 1. Cargar el Modelo y el Tokenizador TTS en español
tts_model_name = "espnet/tts_spanish"
print(f"Cargando modelo TTS: {tts_model_name}")

# Se necesita la versión VITS para este modelo
try:
    tokenizer = AutoTokenizer.from_pretrained(tts_model_name)
    tts_model = VitsModel.from_pretrained(tts_model_name)
except Exception as e:
    print(f"Error al cargar el modelo TTS. Usando un modelo alternativo más ligero: {e}")
    tts_model_name = "facebook/mms-tts-spa"
    tokenizer = AutoTokenizer.from_pretrained(tts_model_name)
    tts_model = VitsModel.from_pretrained(tts_model_name)


# 2. Generar el Audio
text_to_synthesize = corrected_text

# Tokenizar el texto limpio
inputs = tokenizer(text_to_synthesize, return_tensors="pt")

# Generar la salida de audio
with torch.no_grad():
    output = tts_model(**inputs).waveform

# 3. Guardar el Archivo de Audio
output_path = "reconocimiento_a_voz_final.wav"
sampling_rate = tts_model.config.sampling_rate

# Guardar la forma de onda como archivo WAV
sf.write(output_path, output.cpu().numpy().squeeze(), sampling_rate, format='WAV')

print("\n--- RESULTADO TTS ---")
print(f"chivo de audio generado con éxito: '{output_path}'")

# 4. Reproducir el Audio
Audio(output.cpu().numpy().squeeze(), rate=sampling_rate)

Cargando modelo TTS: espnet/tts_spanish


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Error al cargar el modelo TTS. Usando un modelo alternativo más ligero: espnet/tts_spanish is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

--- RESULTADO TTS ---
chivo de audio generado con éxito: 'reconocimiento_a_voz_final.wav'
